In [1]:
# !pip install jiwer -qq

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sk import predict
import onnxruntime
import onnx
from pathlib import Path
import jiwer
import pandas as pd

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
model = onnx.load("/content/neem/onnx/conformer_small.onnx")
model = onnxruntime.InferenceSession(model.SerializeToString())

# Predict single or multiple file

In [5]:
fn = "/content/test/youtube/0228444ff34081eda587c0ca53712486.wav"
preds,files,entropy,timesteps = predict(model,fn)

100%|██████████| 1/1 [00:00<00:00,  7.98it/s]


In [6]:
Path(fn).with_suffix(".txt").read_text()

'pada peringkat ini pembukaan sekolah hanya melibatkan'

In [7]:
preds[0],entropy[0].item()

('pada peringkat ini pembukaan sekolah hanya melibatkan', 0.08249685168266296)

# Predict from single or multiple folder

In [9]:
fn = "/content/test/"
preds,files,entropy,timesteps = predict(model,fn)

100%|██████████| 243/243 [00:40<00:00,  5.99it/s]


In [10]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    data.append([j,label,i,jiwer.compute_measures(label,i)['wer']])

In [11]:
df = pd.DataFrame(data)
df.columns = ["path","label","pred","wer"]
df.head()

,path,label,pred,wer
0,/content/test/audio-iium/29.wav,tahulah kita sama keluarga kita tapi orang dar...,tahulah kita sama keluarga kita tapi orang dar...,0.000000
1,/content/test/audio-iium/49.wav,saya tak salahkan dia orang,saya tak salahkan dia orang,0.000000
2,/content/test/audio-iium/55.wav,benda ni sangat menganggu kehidupan seharian a...,benda ni sangat mengganggu kehidupan seharian ...,0.294118
3,/content/test/audio-iium/64.wav,dah lah suami aku tu bukan calon pilihan,darah suami aku tu bukan calon pilihan,0.250000
4,/content/test/audio-iium/63.wav,dia ni kakak pada mak aku,yia ni kakak pada mak aku,0.166667


In [12]:
# jiwer error rate usually higher than other library, if want to make comparison make sure use same library
# for calculating the error rate.
df["wer"].mean()

0.2531298794432256